In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from astropy.io import fits
import math
import matplotlib.pyplot as plt
from photutils import CircularAnnulus, EllipticalAperture
from astropy.stats import sigma_clip


/tmp/ipykernel_65/1692926063.py:7: DeprecationWarning: `photutils.CircularAnnulus` is a deprecated alias for `photutils.aperture.CircularAnnulus` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAnnulus` to silence this warning.
  from photutils import CircularAnnulus, EllipticalAperture
/tmp/ipykernel_65/1692926063.py:7: DeprecationWarning: `photutils.EllipticalAperture` is a deprecated alias for `photutils.aperture.EllipticalAperture` and will be removed in the future. Instead, please use `from photutils.aperture import EllipticalAperture` to silence this warning.
  from photutils import CircularAnnulus, EllipticalAperture


In [8]:
trainingset = pd.read_csv('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/selectdata.csv')
trainingset.drop(columns=['Unnamed: 0'], inplace=True)

In [10]:
trainingset = trainingset[trainingset['I_auto'] > 21.4]
trainingset = trainingset.sort_values('ref_id')
trainingset = trainingset.reset_index(drop=True)
trainingset

,raw_flux,area,flux,ref_id,I_auto,zp,aperture_x,aperture_y,aperture_theta,aperture_a,aperture_b,path
0,75.160318,32.151808,4.031589,561,21.600,3.954568,842.51416,371.94345,6.1586,0.841363,0.841363,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
1,61.440483,26.954945,2.523411,561,21.600,4.442069,812.50340,3942.46120,6.1586,0.770371,0.770371,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
2,75.258795,43.347160,0.290027,561,21.600,4.073345,742.34600,288.73020,6.1586,0.976925,0.976925,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
3,48.817097,21.572371,2.577795,561,21.600,4.407603,803.77747,351.71930,6.1586,0.689175,0.689175,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
4,61.000378,26.685558,0.557099,561,21.600,4.159620,818.76605,3981.83540,6.1586,0.766512,0.766512,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
...,...,...,...,...,...,...,...,...,...,...,...,...
34554,119.909633,62.545354,6.199727,117165,21.632,4.433446,213.58397,1628.27750,-35.1249,1.411204,0.975814,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
34555,130.416086,76.025207,1.719163,117165,21.632,4.412917,1959.94070,1605.32710,-35.1249,1.513804,1.105731,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
34556,89.250103,39.025751,3.502426,117165,21.632,4.179009,219.52478,1596.80160,-35.1249,1.210977,0.709540,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
34557,79.690630,38.479726,4.391139,117165,21.632,4.035003,263.54697,1598.63500,-35.1249,1.206074,0.702457,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...


In [18]:
#trainingset.to_csv('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/selectdata_reduce_faint.csv')

In [11]:
#generate image
cutout_size = 48

def background_annulus(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    """Measure background in an annulus."""
    
    masked_data = np.ma.array(data=data, mask=mask != 0)
    masked_data = masked_data.filled(fill_value=0)

    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')

    cutout_data = masks.cutout(masked_data)

    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)

    S = pd.Series()
    S['annulus_mean'] = np.ma.mean(clip_annulus_array)
    S['annulus_median'] = np.ma.median(clip_annulus_array)
    S['annulus_std'] = np.ma.std(clip_annulus_array)
    S['annulus_samples'] = np.ma.count(clip_annulus_array)

    return S

def flux_elliptical(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    """Measure the flux withing an elliptical aperture."""
    
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)

    xmask = mask != 0
    raw_flux = aperture_photometry(image, source_aperture, mask=xmask)
   
    S = pd.Series()
    S['raw_flux'] = float(raw_flux['aperture_sum'][0])
    S['area'] = source_aperture.area
    
    return S

def cal_calerror(sig_src,sig_zp,zp,f_src):
    sig_cal = np.sqrt(sig_src**2 * sig_zp**2 + sig_src**2 * zp**2 + sig_zp**2 * f_src**2)
    return sig_cal

def cal_fcal(f_src,zp):
    f_cal = f_src * zp
    return f_cal

def creat_stamps(image, sources):
    y = math.floor(sources['aperture_x'].values[0])
    x = math.floor(sources['aperture_y'].values[0])
    #y = math.floor(sources['aperture_x'])
    #x = math.floor(sources['aperture_y'])
    x_start = max((x - cutout_size), 0)
    x_end = min((x + cutout_size), image.shape[0])
    y_start = max((y - cutout_size), 0)
    y_end = min((y + cutout_size), image.shape[1])

    stamps = image[x_start:x_end, y_start:y_end]
    return stamps

def photometry_oneimage(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    
    S1 = background_annulus(image, mask, aperture_x, aperture_y)
    S2 = flux_elliptical(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b)

    flux_obs = S2['raw_flux'] - S2['area'] * S1['annulus_mean']
    return flux_obs, S1['annulus_std']

def generate_mask(size_data, image):

    num_sample = int(size_data[0] * size_data[1] * (1 - ratio))
    mask = np.ones(size_data)
    output = image

    for ich in range(size_data[2]):
        idy_msk = np.random.randint(0, size_data[0], num_sample)
        idx_msk = np.random.randint(0, size_data[1], num_sample)

        idy_neigh = np.random.randint(-size_window[0] // 2 + size_window[0] % 2, size_window[0] // 2 + size_window[0] % 2, num_sample)
        idx_neigh = np.random.randint(-size_window[1] // 2 + size_window[1] % 2, size_window[1] // 2 + size_window[1] % 2, num_sample)

        idy_msk_neigh = idy_msk + idy_neigh
        idx_msk_neigh = idx_msk + idx_neigh

        idy_msk_neigh = idy_msk_neigh + (idy_msk_neigh < 0) * size_data[0] - (idy_msk_neigh >= size_data[0]) * size_data[0]
        idx_msk_neigh = idx_msk_neigh + (idx_msk_neigh < 0) * size_data[1] - (idx_msk_neigh >= size_data[1]) * size_data[1]

        id_msk = (idy_msk, idx_msk, ich)
        id_msk_neigh = (idy_msk_neigh, idx_msk_neigh, ich)

        output[id_msk] = image[id_msk_neigh]
        mask[id_msk] = 0.0

    return output, mask

In [12]:
#testing blocks

In [13]:
#write the function to generate the mask_annulus, mask_gal
#test the code in the following
def annulus_mask_generator(mask, aperture_x, aperture_y, r_in=30, r_out=45):

    image_shape = (cutout_size*2,cutout_size*2)
    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    
    mask_object = annulus_apertures.to_mask(method='center')
    mask_annulus = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    
    mask_annulus = mask_annulus * (1 - xmask)
    
    return mask_annulus, annulus_apertures.area

def gal_mask_generator(mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    image_shape = (cutout_size*2,cutout_size*2)
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)
    mask_object = source_aperture.to_mask(method='exact')
    mask_image_photutils_fractional = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    mask_gal = mask_image_photutils_fractional * (1 - xmask)
    
    return mask_gal#, source_aperture.area

def background_annulus_jiefeng(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    
    masked_data = np.ma.array(data=data, mask=mask != 0)
    masked_data = masked_data.filled(fill_value=0)

    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')
    cutout_data = masks.cutout(masked_data)
    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)
    background_annulus = np.ma.mean(clip_annulus_array)
    
    return background_annulus

def flux_elliptical_jiefeng(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    image_shape = (cutout_size*2,cutout_size*2)
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)
    mask_object = source_aperture.to_mask(method='exact')
    mask_image_photutils_fractional = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    image_good = image * (1 - xmask)
    
    raw_flux = np.sum(image_good * mask_image_photutils_fractional)#calculate by myself

    background = background_annulus_jiefeng(image, mask, aperture_x, aperture_y)
    
    gal_flux = raw_flux - source_aperture.area * background
    return gal_flux,source_aperture.area

In [16]:
num = 20
row = trainingset.iloc[[num]]

path = row['path'][num]
image = fits.getdata(path)
mask_cutout = fits.getdata(path.replace('.fits', '.mask.fits'))
stamp_image = creat_stamps(image, row)
mask_cutout = creat_stamps(mask_cutout, row)

cutout_size = 48
aperture_x = cutout_size + row['aperture_x'].item() - math.floor(row['aperture_x'].item())
aperture_y = cutout_size + row['aperture_y'].item() - math.floor(row['aperture_y'].item())

mask_gal = gal_mask_generator(mask_cutout, aperture_x, aperture_y,
                                   row['aperture_theta'].item(),row['aperture_a'].item(),row['aperture_b'].item())
mask_annulus, area_annulus = annulus_mask_generator(mask_cutout, aperture_x, aperture_y)
gal,gala_rea=flux_elliptical_jiefeng(stamp_image, mask_cutout, aperture_x, aperture_y, 
                            row['aperture_theta'].item(),row['aperture_a'].item(),row['aperture_b'].item())
print(gal)

4.365206402306725


In [17]:
raw = np.sum(stamp_image*mask_gal*(1-mask_cutout))
bgk = gala_rea*np.sum(stamp_image*mask_annulus*(1-mask_cutout))/area_annulus
print(raw-bgk)

4.319387562161779


In [19]:
#generate the stamp and mask
cutout_size = 48
stamp_train = np.zeros((len(trainingset), cutout_size*2, cutout_size*2))
mask_train = np.zeros((len(trainingset), cutout_size*2, cutout_size*2))

for i in tqdm(range(len(trainingset))):
    row = trainingset.iloc[[i]]
    path = trainingset['path'][i]
    image = fits.getdata(path)
    mask = fits.getdata(path.replace('.fits', '.mask.fits'))
    
    stamp_train[i] = creat_stamps(image, row)
    mask_train[i] = creat_stamps(mask, row)

  4%|▎         | 1250/34559 [01:25<38:09, 14.55it/s] 


KeyboardInterrupt: 

In [ ]:
np.save('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/stamp_trainfaint.npy', stamp_train)
np.save('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_trainfaint.npy', mask_train)

In [ ]:
mask_gal = np.zeros((len(trainingset), cutout_size*2, cutout_size*2))
mask_annulus = np.zeros((len(trainingset), cutout_size*2, cutout_size*2))
area_annulus = np.zeros((len(trainingset)))

for i in tqdm(range(len(trainingset))):
    row = trainingset.iloc[[i]]
    aperture_x = cutout_size + row['aperture_x'].item() - math.floor(row['aperture_x'].item())
    aperture_y = cutout_size + row['aperture_y'].item() - math.floor(row['aperture_y'].item())

    mask_gal[i] = gal_mask_generator(mask_train[i], aperture_x, aperture_y,
                                   row['aperture_theta'].item(),row['aperture_a'].item(),row['aperture_b'].item())
    mask_annulus[i], area_annulus[i] = annulus_mask_generator(mask_train[i], aperture_x, aperture_y)

In [ ]:
np.save('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_galfaint.npy', mask_gal)
np.save('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_annulusfaint.npy', mask_annulus)
np.save('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/area_annulusfaint.npy', area_annulus)